<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [9]:
import cv2
import os
import glob
import tqdm

# Запись процесса работы RAG

In [28]:
data_dir  = os.getcwd() + "/../Data"
i8r_dir = data_dir + '/Res/debug.depth_sample.pointlike'
segm_img_mask = i8r_dir + '/segm_*.png'
list_of_files = filter(os.path.isfile, glob.glob(segm_img_mask))
list_of_files = sorted( list_of_files,
                        key = os.path.getmtime)[::10]

report_dir = data_dir + "/Report"

In [29]:
img_sample = cv2.imread(list_of_files[0])

In [31]:
scale_coef = 1
writer = cv2.VideoWriter(report_dir + '/segm_demonstration.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 90, (img_sample.shape[1] * scale_coef,  img_sample.shape[0] * scale_coef))

for i in tqdm.tqdm(range(0, len(list_of_files), 2)):
    img = cv2.imread(list_of_files[i])
    img = cv2.resize(img, None, fx=scale_coef, fy=scale_coef, interpolation=cv2.INTER_NEAREST)
    writer.write(img)
writer.release() 

100%|██████████| 690/690 [00:02<00:00, 266.43it/s]


# Измерение IoU

In [23]:
import numpy as np
import matplotlib.pyplot as plt 
gt = cv2.imread(data_dir + "/ground_truth.png", cv2.IMREAD_GRAYSCALE)

def IoU(mask1, mask2):
    i_img = cv2.bitwise_and(mask1, mask2)
    u_img = cv2.bitwise_or(mask1, mask2)
    i = np.sum(i_img)
    u = np.sum(u_img) 
    dbg = np.stack((i_img * 0, i_img, u_img * (i_img == 0)), axis=2)
    return i * 1. / u, dbg

In [24]:
ransac_mask = cv2.imread(data_dir + "/RANSAC_mask.png", cv2.IMREAD_GRAYSCALE)
iou_val, iou_debug = IoU(gt, ransac_mask)
print("RANSAC IoU:", iou_val)
cv2.imwrite(report_dir + "/RANSAC_iou_img.png", iou_debug)

RANSAC IoU: 0.7868223638915961


True

In [25]:
rag_mask = cv2.imread(data_dir + "/RAG_mask.png", cv2.IMREAD_GRAYSCALE)
iou_val, iou_debug = IoU(gt, rag_mask)
print("RAG IoU:", iou_val)
cv2.imwrite(report_dir + "/RAG_iou_img.png", iou_debug)

RAG IoU: 0.9881139489194499


True